In [3]:
import json
import pandas as pd
import sys
import wget
from bs4 import BeautifulSoup

In [4]:
path_to_data = 'export (14).csv'

In [5]:
def read_data(path_to_data):
    df = pd.read_csv(path_to_data)
    dff = df.to_json()
    data = json.loads(dff)
    return data

In [6]:
data = read_data(path_to_data)

In [7]:
data.keys()

dict_keys(['title_section', 'ID', 'part', 'concept', 'term'])

In [8]:
data['concept']

{'0': 'http://purl.obolibrary.org/obo/NCIT_C49155',
 '1': 'http://purl.obolibrary.org/obo/NCIT_C49155',
 '2': 'http://sbmi.uth.tmc.edu/ontology/ochv#5636',
 '3': 'http://sbmi.uth.tmc.edu/ontology/ochv#5636',
 '4': 'http://purl.obolibrary.org/obo/NCIT_C88181',
 '5': 'http://purl.obolibrary.org/obo/NCIT_C88181',
 '6': 'http://sbmi.uth.tmc.edu/ontology/ochv#3691',
 '7': 'http://purl.jp/bio/4/id/200906086217709677',
 '8': 'http://sbmi.uth.tmc.edu/ontology/ochv#10490',
 '9': 'http://purl.jp/bio/4/id/200906039727808671',
 '10': 'http://sbmi.uth.tmc.edu/ontology/ochv#C0034394',
 '11': 'http://purl.obolibrary.org/obo/NCIT_C25447',
 '12': 'http://sbmi.uth.tmc.edu/ontology/ochv#15245',
 '13': 'http://sbmi.uth.tmc.edu/ontology/ochv#C0079843',
 '14': 'http://purl.jp/bio/4/id/200906031626509923',
 '15': 'http://purl.obolibrary.org/obo/NCIT_C25755',
 '16': 'http://sbmi.uth.tmc.edu/ontology/ochv#51316',
 '17': 'http://sbmi.uth.tmc.edu/ontology/ochv#3690',
 '18': 'http://sbmi.uth.tmc.edu/ontology/ochv

In [9]:
def get_db_index(data):
    ids = list(set(list(data['ID'].values()))) # whitout nones

    # ART INDEX
    #{
    #   'id':['fst_index','last_index'],
    #   'id':['fst_index','last_index']
    #}
    art_index = {} 

    for i in ids:
        art_index[str(i)] = []

    # get range of index of each title
    for k in data['ID'].keys():
        _id = str(data['ID'][k])  
        if len(art_index[_id]) == 0:
            art_index[_id].append(k)
        elif len(art_index[_id]) == 1:
            art_index[_id].append(k)
        elif len(art_index[_id]) == 2:
            art_index[_id][-1] = k
    return art_index

In [10]:
art_index = get_db_index(data)

In [11]:
art_index

{'33095680.0': ['185192', '185337'],
 '15761412.0': ['628837', '628888'],
 '33161221.0': ['807463', '807538'],
 '32210950.0': ['682263', '682461'],
 '33456134.0': ['862417', '862472'],
 '32768012.0': ['206279', '206359'],
 '25559054.0': ['444695', '444750'],
 '33259536.0': ['293147', '293166'],
 '32309267.0': ['895626', '895650'],
 '32047128.0': ['254269', '254507'],
 '16580632.0': ['787590', '787641'],
 '32047134.0': ['92463', '92663'],
 '25559071.0': ['79825', '79917'],
 '31752224.0': ['727352', '727410'],
 '32833573.0': ['164726', '164771'],
 '33062953.0': ['938562', '938676'],
 '20217898.0': ['452495', '452652'],
 '16154667.0': ['936943', '936998'],
 '33128497.0': ['336809', '336858'],
 '33390643.0': ['595088', '595117'],
 '32800824.0': ['75872', '75902'],
 '33587262.0': ['708903', '708953'],
 '33292351.0': ['284616', '284645'],
 '23953475.0': ['862349', '862395'],
 '28803142.0': ['483289', '483467'],
 '33521739.0': ['674264', '674286'],
 '33292368.0': ['461460', '461493'],
 '33456

In [12]:
art_index = {'33095680.0': ['185192', '185337'],
             '15761412.0': ['628837', '628888'],
             '33161221.0': ['807463', '807538'],
             '32210950.0': ['682263', '682461'],
             '33456134.0': ['862417', '862472']}

In [13]:
def generate_json_ann_counts_db(art_index):
    db = []
    for i in art_index.keys():
        #print(i)
        _id = i
        ann = {'id':_id,
              'terms':[],
              'onts':[]
              }

        _range = art_index[_id]
        
        sections = []

        terms = []
        
        ontos = []

        all_all = []

        for i in range(int(_range[0]),int(_range[-1])+1):
            print(i)

            title_section = data['title_section'][str(i)]

            try:

                ontology = data['part'][str(i)].split('/')[-1]

            except AttributeError:

                ontology = 'None'

            clase = data['concept'][str(i)]

            termino = data['term'][str(i)]
            
            sections.append(title_section)

            terms.append(termino)

            ontos.append(ontology)

            _all = [termino, title_section, ontology, clase]

            all_all.append(_all)
        
        # sections
        unique_sects = list(set(sections))
        sects = []
        for i in unique_sects:
            sects.append(
            {
                'section':i,
                'features':[]
            })
            
        for i in sects:
            for j in all_all:
                if i['section'] == j[1]:
                    i['features'].append({
                        'term': j[0],
                        'ontology':j[2],
                        'class': j[3]
                    })
        
        
        # terms
        unique_terms = list(set(terms))
        terms = []
        for i in unique_terms:
            terms.append({
                    'term':i,
                    'abs_count':0,
                    'features':[]

                })

        for i in terms:
            for j in all_all:
                if i['term'] == j[0]:
                    i['abs_count'] += 1
                    i['features'].append(
                        {
                            'section':j[1],
                            'ontology':j[2],
                            'class':j[3]
                        })           

        # ontos
        unique_ontos = list(set(ontos))
        ontos = []
        for i in unique_ontos:
            ontos.append({
                    'ontology':i,
                    'abs_count':0,
                    'features':[]

                })

        for i in ontos:
            for j in all_all:
                if i['ontology'] == j[2]:
                    i['abs_count'] += 1
                    i['features'].append(
                        {
                            'section':j[1],
                            'term':j[0],
                            'class':j[3]
                        })
        ann['sections'] = sects
        ann['terms'] = terms
        ann['onts'] = ontos
        db.append(ann)
    return db

In [14]:
art_index

{'33095680.0': ['185192', '185337'],
 '15761412.0': ['628837', '628888'],
 '33161221.0': ['807463', '807538'],
 '32210950.0': ['682263', '682461'],
 '33456134.0': ['862417', '862472']}

In [15]:
db = generate_json_ann_counts_db(art_index)

185192
185193
185194
185195
185196
185197
185198
185199
185200
185201
185202
185203
185204
185205
185206
185207
185208
185209
185210
185211
185212
185213
185214
185215
185216
185217
185218
185219
185220
185221
185222
185223
185224
185225
185226
185227
185228
185229
185230
185231
185232
185233
185234
185235
185236
185237
185238
185239
185240
185241
185242
185243
185244
185245
185246
185247
185248
185249
185250
185251
185252
185253
185254
185255
185256
185257
185258
185259
185260
185261
185262
185263
185264
185265
185266
185267
185268
185269
185270
185271
185272
185273
185274
185275
185276
185277
185278
185279
185280
185281
185282
185283
185284
185285
185286
185287
185288
185289
185290
185291
185292
185293
185294
185295
185296
185297
185298
185299
185300
185301
185302
185303
185304
185305
185306
185307
185308
185309
185310
185311
185312
185313
185314
185315
185316
185317
185318
185319
185320
185321
185322
185323
185324
185325
185326
185327
185328
185329
185330
185331
185332
185333
185334

In [16]:
len(db)

5

In [17]:
db[0]['sections']

[{'section': 'Role of Chest CT for Prognostication',
  'features': [{'term': 'CHEST CT',
    'ontology': 'id',
    'class': 'http://purl.jp/bio/4/id/200906069720605992'},
   {'term': 'CHEST CT',
    'ontology': 'ochv',
    'class': 'http://sbmi.uth.tmc.edu/ontology/ochv#20113'},
   {'term': 'CHEST CT',
    'ontology': 'ochv',
    'class': 'http://sbmi.uth.tmc.edu/ontology/ochv#C0202823'},
   {'term': 'CHEST',
    'ontology': 'id',
    'class': 'http://purl.jp/bio/4/id/kb0000000770'},
   {'term': 'CHEST',
    'ontology': 'ochv',
    'class': 'http://sbmi.uth.tmc.edu/ontology/ochv#C0817096'},
   {'term': 'CHEST',
    'ontology': 'ochv',
    'class': 'http://sbmi.uth.tmc.edu/ontology/ochv#49411'},
   {'term': 'CHEST',
    'ontology': 'UBERON',
    'class': 'http://purl.obolibrary.org/obo/UBERON_0001443'},
   {'term': 'CHEST',
    'ontology': 'NCIT',
    'class': 'http://purl.obolibrary.org/obo/NCIT_C25389'}]},
 {'section': 'Reporting and Communicating Chest CT Findings',
  'features': [{'

In [19]:
db[0]['terms']

[{'term': 'SEVERE',
  'abs_count': 3,
  'features': [{'section': 'Role of Chest CT in Diagnostic Decision Making',
    'ontology': 'ochv',
    'class': 'http://sbmi.uth.tmc.edu/ontology/ochv#20290'},
   {'section': 'Role of Chest CT in Diagnostic Decision Making',
    'ontology': 'NCIT',
    'class': 'http://purl.obolibrary.org/obo/NCIT_C70667'},
   {'section': 'Role of Chest CT in Diagnostic Decision Making',
    'ontology': 'ochv',
    'class': 'http://sbmi.uth.tmc.edu/ontology/ochv#C0205082'}]},
 {'term': 'COVID-19',
  'abs_count': 6,
  'features': [{'section': 'Chest CT of COVID-19 Complications',
    'ontology': 'COVIDO',
    'class': 'http://purl.obolibrary.org/obo/COVIDO_0001141'},
   {'section': 'WHO Case Definitions of COVID-19 and RT-PCR Tests',
    'ontology': 'COVIDO',
    'class': 'http://purl.obolibrary.org/obo/COVIDO_0001141'},
   {'section': 'Chest CT of COVID-19 Complications',
    'ontology': 'COVIDO',
    'class': 'http://purl.obolibrary.org/obo/COVIDO_0001141'},
   

In [20]:
db[0]['onts']

[{'ontology': 'NCIT',
  'abs_count': 23,
  'features': [{'section': 'WHO Case Definitions of COVID-19 and RT-PCR Tests',
    'term': 'CASE',
    'class': 'http://purl.obolibrary.org/obo/NCIT_C49152'},
   {'section': 'Role of Chest CT in Diagnostic Decision Making',
    'term': 'BODY',
    'class': 'http://purl.obolibrary.org/obo/NCIT_C13041'},
   {'section': 'Role of Chest CT in Diagnostic Decision Making',
    'term': 'SEVERE',
    'class': 'http://purl.obolibrary.org/obo/NCIT_C70667'},
   {'section': 'Role of Chest CT in Diagnostic Decision Making',
    'term': 'DECISION',
    'class': 'http://purl.obolibrary.org/obo/NCIT_C53285'},
   {'section': 'Chest CT Appearance of COVID-19',
    'term': 'INCIDENCE',
    'class': 'http://purl.obolibrary.org/obo/NCIT_C16726'},
   {'section': 'Chest CT Appearance of COVID-19',
    'term': 'INCIDENCE',
    'class': 'http://purl.obolibrary.org/obo/NCIT_C16726'},
   {'section': 'Chest CT Appearance of COVID-19',
    'term': 'INCIDENCE',
    'class': 

In [ ]:
with open('conteo_ann_art.json','w') as jh:
    jh.write(json.dumps(db))

In [11]:
with open('conteo_ann_art.json','r') as fh:
    ann = json.loads(fh.read())

In [ ]:
ann={
    'id':'int',
    'sections':[{
                    'section':'str',
                    'features':[
                                    {
                                        'term':'str',
                                        'ontology':'str',
                                        'class':'str'
                                    },...,
                                    N
                                ]
                },...,

            N],
    
    'terms':[{
                    'term':'str',
                    'abs_count':'int',
                    'features':[
                                    {
                                        'section':'str',
                                        'ontology':'str',
                                        'class':'str'
                                    },...,
                                    N
                                ]
                },...,

            N],
            
    'ontos':[{
                    'ontology':'str',
                    'abs_count':'int',
                    'features':[
                                    {
                                        'section':'str',
                                        'class':'str'
                                    }
                               ]
                },...,
            N]}   
        
# Estructura de datos a nivel de base de datos

#db = [ann1,...,annN]

In [12]:
ann[0]

{'id': '33095680.0',
 'terms': [{'term': 'PATIENTS',
   'abs_count': 6,
   'features': [{'section': 'Role of Chest CT in Diagnostic Decision Making',
     'ontology': 'ochv',
     'class': 'http://sbmi.uth.tmc.edu/ontology/ochv#9372'},
    {'section': 'Role of Chest CT in Diagnostic Decision Making',
     'ontology': 'ochv',
     'class': 'http://sbmi.uth.tmc.edu/ontology/ochv#9372'},
    {'section': 'Role of Chest CT in Diagnostic Decision Making',
     'ontology': 'ochv',
     'class': 'http://sbmi.uth.tmc.edu/ontology/ochv#9372'},
    {'section': 'Role of Chest CT in Diagnostic Decision Making',
     'ontology': 'NCIT',
     'class': 'http://purl.obolibrary.org/obo/NCIT_C16960'},
    {'section': 'Role of Chest CT in Diagnostic Decision Making',
     'ontology': 'NCIT',
     'class': 'http://purl.obolibrary.org/obo/NCIT_C16960'},
    {'section': 'Role of Chest CT in Diagnostic Decision Making',
     'ontology': 'NCIT',
     'class': 'http://purl.obolibrary.org/obo/NCIT_C16960'}]},
  

In [ ]:
data[0].keys()

In [ ]:
URL_FSERVER = 'http://caribe.udea.edu.co/pmc_murcia/'

In [ ]:
def ReadFileServer():
    
    wget.download(URL_FSERVER, out='./article_records/Findex.html')
    
    with open('./article_records/Findex.html','r') as htmlFile:
        html = htmlFile.read()
        
    soup = BeautifulSoup(html)
    
    pmcs = [tag['href'] for tag in soup.findAll('a', href=True) if 'PMC' in tag['href']]
    
    return pmcs

In [ ]:
pmcs=ReadFileServer()

In [ ]:
def DownLoadRecords(): 
    records = ReadFileServer()
    print('Server conted read; Num of files:',len(records))
    for r in records:
        wget.download(URL_FSERVER+r,out='./article_records/'+r)